<a href="https://colab.research.google.com/github/WayneGretzky1/nba_game_predictor/blob/main/nba_game_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
nba_df = pd.read_csv("https://raw.githubusercontent.com/WayneGretzky1/nba_game_predictor/refs/heads/main/cleaned.csv")
nba_df

,gameid,gamedate,home_teamid,away_teamid,home_win,away_win,home_win_avg_prev5,away_win_avg_prev5,home_points_avg_prev5,away_points_avg_prev5,...,home_freethrowsattempted_avg_prev5,away_freethrowsattempted_avg_prev5,home_freethrowspercentage_avg_prev5,away_freethrowspercentage_avg_prev5,home_reboundstotal_avg_prev5,away_reboundstotal_avg_prev5,home_foulspersonal_avg_prev5,away_foulspersonal_avg_prev5,home_turnovers_avg_prev5,away_turnovers_avg_prev5
0,22100127,2021-11-05,1610612761,1610612739,False,True,0.4,0.6,102.8,99.4,...,21.2,22.8,0.7074,0.7750,46.2,47.6,20.0,16.0,14.4,17.4
1,22100137,2021-11-06,1610612756,1610612737,True,False,0.6,0.2,108.2,102.0,...,21.8,16.0,0.7802,0.7342,48.2,50.6,17.8,17.2,15.0,13.2
2,22100145,2021-11-07,1610612744,1610612745,True,False,0.8,0.0,111.8,102.6,...,14.8,23.8,0.6820,0.6300,51.8,46.4,18.2,19.4,17.6,21.0
3,22100150,2021-11-08,1610612742,1610612740,True,False,0.8,0.2,108.4,96.0,...,20.6,18.6,0.6490,0.8328,44.2,49.2,20.0,18.4,12.6,20.8
4,22100154,2021-11-08,1610612747,1610612766,True,False,0.8,0.2,110.4,105.4,...,16.4,20.8,0.7542,0.7388,42.6,39.6,18.8,17.0,16.8,13.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4669,22401195,2025-04-13,1610612750,1610612762,True,False,0.8,0.0,122.6,110.0,...,25.8,22.2,0.8206,0.8264,45.2,41.2,18.6,18.0,11.8,11.4
4670,22401191,2025-04-13,1610612755,1610612741,False,True,0.0,0.6,107.2,127.2,...,16.4,22.0,0.7026,0.8088,40.4,42.8,17.0,17.4,13.8,14.2
4671,22401200,2025-04-13,1610612758,1610612756,True,False,0.2,0.0,108.8,105.8,...,16.8,17.0,0.8130,0.7944,42.0,38.6,19.2,19.8,12.8,12.2
4672,22401187,2025-04-13,1610612738,1610612766,True,False,0.8,0.0,116.8,97.2,...,15.6,16.8,0.7824,0.7564,50.8,47.8,12.8,14.2,11.2,17.0


In [ ]:
#choose features X and label y
X = nba_df.drop(columns=['home_win', 'away_win', 'gameid', 'gamedate', 'home_teamid', 'away_teamid']).to_numpy() #every col except these
y = nba_df['home_win'].to_numpy()

In [ ]:
#train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#normalize with standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
# print(X_train_norm)

In [ ]:
#PyTorch Dataset class
class NumpyDataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, input_array, target_output_array):
    'Initialization'
    self.input_tensor = torch.from_numpy(input_array).float()
    self.output_tensor = torch.from_numpy(target_output_array).float()

  def __len__(self):
      'Denotes the total number of samples'
      return len(self.input_tensor)

  def __getitem__(self, index):
      'Generates one sample of data'
      # Load data and get label
      X = self.input_tensor[index]
      y = self.output_tensor[index]

      return X, y

In [ ]:
#Convert to PyTorch Dataset
training_dataset = NumpyDataset(X_train_norm, y_train)
testing_dataset = NumpyDataset(X_test_norm, y_test)


In [ ]:
import matplotlib.pyplot as plt
#Train
def train(model,  data_generator, loss_function, optimizer):
  model.train() #updates any model layers that behave differently in training and execution
  avg_loss = 0
  num_batches = 0
  for i, (input_data, target_output) in enumerate(data_generator):
    optimizer.zero_grad()                            # Gradients need to be reset each batch
    prediction = model(input_data).flatten()                   # Forward pass: compute the output class given a image

    loss = loss_function(prediction, target_output)  # Compute the loss: difference between the output and correct result
    loss.backward()                                  # Backward pass: compute the gradients of the model with respect to the loss
    optimizer.step()
    avg_loss += loss.item()
    num_batches += 1
  return avg_loss/num_batches

#Test
def test(model, test_loader, loss_function):
  model.eval() #updates any model layers that behave differently in training and execution
  test_loss = 0
  num_batches = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data).flatten()
      test_loss += loss_function(output, target).item()
      pred = output.data.max()
      num_batches += 1
  test_loss /= num_batches
  return test_loss

#Log Results
def logResults(epoch, num_epochs, train_loss, train_loss_history, test_loss, test_loss_history, epoch_counter, print_interval=1000):
  if (epoch%print_interval == 0):  print('Epoch [%d/%d], Train Loss: %.4f, Test Loss: %.4f' %(epoch+1, num_epochs, train_loss, test_loss))
  train_loss_history.append(train_loss)
  test_loss_history.append(test_loss)
  epoch_counter.append(epoch)

#Graph the loss
def graphLoss(epoch_counter, train_loss_hist, test_loss_hist, loss_name="Loss", start = 0):
  fig = plt.figure()
  plt.plot(epoch_counter[start:], train_loss_hist[start:], color='blue')
  plt.plot(epoch_counter[start:], test_loss_hist[start:], color='red')
  plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
  plt.xlabel('#Epochs')
  plt.ylabel(loss_name)

#Combine
def trainAndGraph(model, training_generator, testing_generator, loss_function, optimizer, num_epochs, learning_rate, logging_interval=1):
  #Arrays to store training history
  test_loss_history = []
  epoch_counter = []
  train_loss_history = []

  for epoch in range(num_epochs):
    avg_loss = train(model, training_generator, loss_function, optimizer)
    test_loss = test(model, testing_generator, loss_function)
    logResults(epoch, num_epochs, avg_loss, train_loss_history, test_loss, test_loss_history, epoch_counter, logging_interval)

  graphLoss(epoch_counter, train_loss_history, test_loss_history)

In [ ]:
class LinearClassifier(nn.Module):
    def __init__(self, features, numClasses):
        super().__init__()
        self.linear1 = nn.Linear(features, numClasses)

    def forward(self, x):
        x = self.linear1(x)
        return x

In [ ]:
#Compute accuracy of model
def computeLabelAccuracy(networkModel, data_loader, label_text=""):
  correct = 0
  total = 0
  with torch.no_grad():
    for data, target in data_loader:
      output = networkModel(data).flatten()
      probs = torch.sigmoid(output) #get the probabilities of each output from the logit
      correct += ((probs > 0.5) == target).sum().item()
      total += target.size(0)
  print('\n{}: Accuracy: {}/{} ({:.1f}%)'.format(
    label_text, correct, len(data_loader.dataset),
    100. * correct / len(data_loader.dataset)))
  return 100. * correct / len(data_loader.dataset)

In [ ]:
#Tune epochs, learning rate, batch size, and weight decay
def LogisticRegressionModel(num_epochs, lr, batch_size, wd, train, test):
  predModel = LinearClassifier(X_train_norm.shape[1],1)

  model = predModel;
  optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=wd) #weight decays [0.1, 0.01, 0.001, ]
  loss_function = torch.nn.BCEWithLogitsLoss()

  training_generator = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
  testing_generator = torch.utils.data.DataLoader(test, batch_size=batch_size)

  print("Loaded Data")

  trainAndGraph(model, training_generator, testing_generator, loss_function, optimizer, num_epochs, lr, 10000)
  train_acc = computeLabelAccuracy(model, training_generator, "Training")
  test_acc = computeLabelAccuracy(model, testing_generator, "Testing")

  return train_acc, test_acc

In [ ]:
#Hyperparameters to test
lrs = [0.1, 0.01, 0.001, 0.0001, 0.00001]
decs = [0.1, 0.01, 0.001, 0.0001, 0.00001]
batch_sizes = [50, 100, 200, 300, 400, 500]
epochs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150]
from sklearn.model_selection import KFold, cross_val_score

#randomized search with cross validation
import random
all_avg_accs = [] #avg accuracy of each iteration
iter = 0
for i in range(25):
  lr = random.choice(lrs)
  dec = random.choice(decs)
  bs = random.choice(batch_sizes)
  e = random.choice(epochs)

  #IMPLEMENT CROSS VAL
  kf = KFold(n_splits=5, shuffle=True, random_state=42)
  scores = [] #each accuracy
  for train_idx, test_idx in kf.split(X):
    x_train2 = X[train_idx]
    y_train2 = y[train_idx]
    x_test2 = X[test_idx]
    y_test2 = y[test_idx]
    x_dataset = NumpyDataset(x_train2, y_train2)
    y_dataset = NumpyDataset(x_test2, y_test2)
    train_acc, test_acc = LogisticRegressionModel(e,lr, bs, dec, x_dataset, y_dataset)
    scores.append(test_acc)
    print(test_acc)
  iter += 1
  avg_acc = np.mean(scores) #avg accuracy of the iteration
  all_avg_accs.append({"iter": iter, "avg_acc": avg_acc, "e": e, "lr": lr, "bs": bs, "wd": dec})
  # print(f"iteration {iter} with e: {e}, lrd: {lr}, bs: {bs}, dec: {dec}")
for acc in all_avg_accs:
  print(acc)

In [ ]:
#Use the tuned hyperparameters
predModel = LinearClassifier(X_train_norm.shape[1],1)

model = predModel;
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.01)
loss_function = torch.nn.BCEWithLogitsLoss()

training_generator = torch.utils.data.DataLoader(training_dataset, batch_size=500, shuffle=True)
testing_generator = torch.utils.data.DataLoader(testing_dataset, batch_size=500)

print("Loaded Data")

trainAndGraph(model, training_generator, testing_generator, loss_function, optimizer, 150, 0.01, 10)
train_acc = computeLabelAccuracy(model, training_generator, "Training")
test_acc = computeLabelAccuracy(model, testing_generator, "Testing")

In [ ]:
from sklearn import metrics
preds = model(torch.tensor(X_test_norm).float())
# print(preds)
probs = torch.sigmoid(preds).detach().numpy() #convert to probabilities
# print(probs)
y_pred = (probs >= 0.5)
# print(X_test.shape)
# print(y_test.shape)
# print(preds)
fpr, tpr, thresholds = metrics.roc_curve(y_test,probs)
plt.plot(fpr,tpr)
plt.title("ROC Curve")
print("AUC:",metrics.auc(fpr, tpr))
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
p,r,f,count = metrics.precision_recall_fscore_support(y_test, y_pred)
print("Precision",p)
print("Recall",r)
print("F1",f)
print("Count",count)

In [ ]:
#pre-trained model EC
import huggingface_hub
huggingface_hub.login()
!pip install tabpfn
!pip install --upgrade tabpfn
#implement pre-trained model for extra credit
from sklearn.metrics import accuracy_score
from tabpfn import TabPFNClassifier

clf = TabPFNClassifier(ignore_pretraining_limits=True)  # Uses TabPFN 2.5 weights, finetuned on real data.
clf.fit(X_train_norm, y_train)

prediction_probabilities = clf.predict_proba(X_test_norm)
preds = clf.predict(X_test_norm)
print("Accuracy", accuracy_score(y_test, predictions))